In [1]:
CONFIG = {
    'data_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/data/',
    'models_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/',
    'results_folder': 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/',

    'test_path': 'test_pairs.csv',  
    'model_path': '6.2.pth', 
    'output_path': '6.2_predictions.csv',
    'items_meta_path' : 'items_meta.parquet',
    'users_meta_path' : 'users_meta.parquet',

    'user_emb_size' : 512, # 183404
    'item_emb_size' : 512, # 337727
    'source_emb_size' : 128, # 19613
    'age_emb_size' : 4, # 43
    'duration_emb_size' : 16, # ~175   
    'gender_emb_size' : 4, # 2
    'torch_precision' : 40, # number of decimal places for printing numbers

    'DEVICE': 'cuda',
    'SEED': 42,
    'BATCH_SIZE': 16384,
    'output_dim': 3
}

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm

In [3]:
device = torch.device(CONFIG['DEVICE'] if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=CONFIG['torch_precision'])

torch.manual_seed(CONFIG['SEED'])
torch.cuda.manual_seed_all(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

In [4]:
test = pd.read_csv(f"{CONFIG['data_folder']}{CONFIG['test_path']}")

items_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
items_meta['duration'] = items_meta['duration'] - 5
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta['duration'] = items_meta['duration'].astype('category')
items_meta.set_index('item_id', inplace=True)

users_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['age'] = users_meta['age'] - 18
users_meta['gender'] = users_meta['gender'].replace({1:0, 2:1})
users_meta['user_id'] = users_meta['user_id'].astype('category')
users_meta['gender'] = users_meta['gender'].astype('category')
users_meta['age'] = users_meta['age'].astype('category')
users_meta.set_index('user_id', inplace=True)

In [5]:
class AFMModel(nn.Module):
    def __init__(self, 
                 input_dim, 
                 num_users=users_meta.index.nunique(), 
                 num_items=items_meta.index.nunique(), 
                 num_sources=items_meta['source_id'].nunique(),
                 num_ages=users_meta['age'].nunique(),
                 num_durations=items_meta['duration'].nunique(),
                 num_genders=users_meta['gender'].nunique(), 
                 output_dim=CONFIG['output_dim']): 
        super(AFMModel, self).__init__()
        
        # Embedding layers for each feature
        self.user_embedding = nn.Embedding(num_users, CONFIG['user_emb_size'])
        self.item_embedding = nn.Embedding(num_items, CONFIG['item_emb_size'])
        self.source_embedding = nn.Embedding(num_sources, CONFIG['source_emb_size'])
        self.age_embedding = nn.Embedding(num_ages, CONFIG['age_emb_size'])
        self.duration_embedding = nn.Embedding(num_durations, CONFIG['duration_emb_size'])
        self.gender_embedding = nn.Embedding(num_genders, CONFIG['gender_emb_size'])
        
        # Transformations for embeddings
        self.user_emb_transform = nn.Linear(CONFIG['user_emb_size'], 32)
        self.item_emb_transform = nn.Linear(CONFIG['item_emb_size'], 32)
        self.source_emb_transform = nn.Linear(CONFIG['source_emb_size'], 32)
        self.age_emb_transform = nn.Linear(CONFIG['age_emb_size'], 32)
        self.duration_emb_transform = nn.Linear(CONFIG['duration_emb_size'], 32)
        self.gender_emb_transform = nn.Linear(CONFIG['gender_emb_size'], 32)
        
        # Attention layer components
        self.attention_network = nn.Sequential(
            nn.Linear(64, 32),  # 64 comes from the concatenation of two embedding sizes
            nn.GELU(),
            nn.Linear(32, 1)
        )

        self.output_layer = nn.Linear(32, output_dim)  # Output layer
    
    def pairwise_interaction(self, x1, x2):
        return x1 * x2  # Element-wise multiplication for feature interaction
    
    def compute_attention(self, emb1, emb2):
        # Concatenate embeddings of the pair of features and pass through attention network
        combined_emb = torch.cat([emb1, emb2], dim=-1)  # Concatenate embeddings (dim=-1 is feature dimension)
        attention_score = self.attention_network(combined_emb)
        attention_score = torch.sigmoid(attention_score)  # Use sigmoid for attention score
        return attention_score
    
    def forward(self, user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids, embeddings):
        # Obtain embeddings for all features
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        source_emb = self.source_embedding(source_ids)
        age_emb = self.age_embedding(age_ids)
        duration_emb = self.duration_embedding(duration_ids)
        gender_emb = self.gender_embedding(gender_ids)
        
        # Apply linear transformations to embeddings
        user_emb = self.user_emb_transform(user_emb)
        item_emb = self.item_emb_transform(item_emb)
        source_emb = self.source_emb_transform(source_emb)
        age_emb = self.age_emb_transform(age_emb)
        duration_emb = self.duration_emb_transform(duration_emb)
        gender_emb = self.gender_emb_transform(gender_emb)
        
        # Create dictionary of all embeddings
        embeddings_dict = {
            'user': user_emb,
            'item': item_emb,
            'source': source_emb,
            'age': age_emb,
            'duration': duration_emb,
            'gender': gender_emb,
            'embeddings': embeddings
        }
        
        interactions = []
        attention_scores = []
        
        # Compute pairwise interactions and their attention scores
        for key1, emb1 in embeddings_dict.items():
            for key2, emb2 in embeddings_dict.items():
                if key1 < key2:  # Avoid double counting pairs (e.g., user-item and item-user)
                    interaction = self.pairwise_interaction(emb1, emb2)
                    attention_score = self.compute_attention(emb1, emb2)
                    interactions.append(interaction * attention_score)  # Weight the interaction by attention score
                    attention_scores.append(attention_score)  # Store the attention scores
        
        # Combine all interactions into a single tensor
        interactions = torch.stack(interactions, dim=1)  # Shape: [batch_size, num_interactions]
        attention_scores = torch.stack(attention_scores, dim=1)  # Shape: [batch_size, num_interactions]
        
        # Sum over all interactions with their respective attention scores
        weighted_interactions = torch.sum(interactions * attention_scores, dim=1)
        
        # Pass through output layer to get the final prediction
        output = self.output_layer(weighted_interactions)
        
        return output

In [6]:
input_dim = (CONFIG['user_emb_size'] + 
             CONFIG['item_emb_size'] + 
             CONFIG['source_emb_size'] + 
             CONFIG['age_emb_size'] +
             CONFIG['duration_emb_size'] + 
             CONFIG['gender_emb_size'] + 
             32)

In [7]:
model = AFMModel(input_dim).to(device)
model.load_state_dict(torch.load(f"{CONFIG['models_folder']}{CONFIG['model_path']}")['model_state_dict'])
model.eval()

C:\Users\Николай\AppData\Local\Temp\ipykernel_15348\2852703818.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder'

AFMModel(
  (user_embedding): Embedding(183404, 512)
  (item_embedding): Embedding(337727, 512)
  (source_embedding): Embedding(19613, 128)
  (age_embedding): Embedding(43, 4)
  (duration_embedding): Embedding(176, 16)
  (gender_embedding): Embedding(2, 4)
  (user_emb_transform): Linear(in_features=512, out_features=32, bias=True)
  (item_emb_transform): Linear(in_features=512, out_features=32, bias=True)
  (source_emb_transform): Linear(in_features=128, out_features=32, bias=True)
  (age_emb_transform): Linear(in_features=4, out_features=32, bias=True)
  (duration_emb_transform): Linear(in_features=16, out_features=32, bias=True)
  (gender_emb_transform): Linear(in_features=4, out_features=32, bias=True)
  (attention_network): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
  (output_layer): Linear(in_features=32, out_features=3, bias=True)
)

In [8]:
predictions = []
num_samples = len(test)
num_batches = (num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

with torch.no_grad():
    for batch_idx in tqdm(range(num_batches), desc="Predicting", unit="batch"):
        start_idx = batch_idx * CONFIG['BATCH_SIZE']
        end_idx = min(start_idx + CONFIG['BATCH_SIZE'], num_samples)
        batch_main = test.iloc[start_idx:end_idx]
        batch_user_ids = batch_main['user_id'].values
        batch_item_ids = batch_main['item_id'].values
        
        batch_users_meta = users_meta.loc[batch_user_ids]
        batch_items_meta = items_meta.loc[batch_item_ids]
        
        batch_user_values = torch.tensor(batch_user_ids, dtype=torch.long, device=device)
        batch_item_values = torch.tensor(batch_item_ids, dtype=torch.long, device=device)
        
        batch_gender_values = torch.tensor(batch_users_meta['gender'].values, dtype=torch.long, device=device)
        batch_age_values = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
        batch_source_values = torch.tensor(batch_items_meta['source_id'].values, dtype=torch.long, device=device)
        batch_duration_values = torch.tensor(batch_items_meta['duration'].values, dtype=torch.long, device=device)

        embeddings = torch.tensor(np.stack(batch_items_meta['embeddings'].values), device=device, dtype=torch.float32)

        outputs = model(batch_user_values, batch_item_values, batch_source_values, batch_age_values, batch_duration_values, batch_gender_values, embeddings)
        probabilities = F.softmax(outputs, dim=1)

        # Взвешенные предсказания
        class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
        weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()

        predictions.extend(weighted_predictions)

Predicting: 100%|██████████| 102/102 [00:08<00:00, 12.05batch/s]


In [9]:
# Добавление предсказанных значений в DataFrame
test['predict'] = predictions

test.to_csv(f"{CONFIG['results_folder']}{CONFIG['output_path']}", index=False)
print(f"Predictions saved to f'{CONFIG['results_folder']}{CONFIG['output_path']}'")

Predictions saved to f'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/6.2_predictions.csv'
